# Fetch the Ethereum on-chain data via web3 package

- Author: Yichen Luo
- Date: 2024-09-30

## Dependencies

- Install the `web3` library using the following command:
```bash
pip install web3
```

In [1]:
import json
from typing import Any, Optional

from web3 import HTTPProvider, Web3

from environ.constants import DATA

## Connect to an Ethereum node: Infura

- Infura is a popular Ethereum node provider. You can sign up for a free account at [Infura](https://www.infura.io/).

<img src="./fig/config.png" width="1400">

- Copy the api key and save it in a secure place

<img src="./fig/api.png" width="1400">

In [3]:
# Remember to set your Infura API key
INFURA_API_KEY = "YOUR_API_KEY"
infuraurl = f'https://mainnet.infura.io/v3/{INFURA_API_KEY}'
w3 = Web3(HTTPProvider(infuraurl))

## Check the contract address of contract of interest

Let's take the Compound CToken contract as an example. Compound is a decentralized, blockchain-based protocol that allows you to lend and borrow cryptocurrencies. The Compound protocol is implemented as a set of Ethereum smart contracts. The Compound website is [here](https://compound.finance/).

- Step 1: Use your wallet to interact with the Compound protocol. 

<img src="./fig/compound.png" width="1400">

- Step 2: Go to the market page and find the asset of interest. For example, we are interested in the cUSDC market.
 
<img src="./fig/market.png" width="1400">

- Step 3: Click the `View on Etherscan` button to find the contract address.

<img src="./fig/market_data.png" width="1400">

- Step 4: Copy the contract address and click the `Contract` button.

<img src="./fig/etherscan.png" width="1400">

- Step 5: Check whether there is an implementation contract. If there is, click the implementation contract.

<img src="./fig/implementation.png" width="1400">

- Step 6: Download the ABI in Json for the implementation contract (if there is an implementation contract. if not, abi of original contract).

<img src="./fig/abi.png" width="1400">

In [4]:
# address of the original contract
address = "0xc3d688B66703497DAA19211EEdff47f25384cdc3"

- Step 7: Back to the original contract page and click the `Read as Proxy` button to find the read-only function of interest.

<img src="./fig/func.png" width="1400">

- Step 8: Construct the function using web3 and ethtools.

In [5]:
class FunctionCaller:
    """
    Class to call functions from the smart contract
    """

    def __init__(self, contract_address: str, w3: Web3, abi_path: str):
        self.contract_address = contract_address
        self.w3 = w3
        self.abi_path = abi_path

    def _load_abi(
        self
    ) -> dict:
        """
        Function to load the abi of the smart contract

        Args:
            path (str): The path to the abi file

        Returns:    
            dict: The abi of the smart contract
        """

        with open(self.abi_path, "r", encoding="utf-8") as f:
            abi = json.load(f)

        return abi

    def _get_contract(
        self
    ) -> Any:
        """
        Function to get the contract object

        Returns:
            Any: The contract object
        """
        abi = self._load_abi()
        contract = self.w3.eth.contract(address=self.contract_address, abi=abi)
        return contract

    def call_function(
        self,
        function_name: str,
        block_identifier: int | str = "latest",
        params: Optional[Any] = None,
    ):
        """
        Function to call a function from the smart contract

        Args:
            function_name (str): The name of the function to call
            block_identifier (int | str, optional): The block number or 
                block hash to call the function at (default is 'latest').
            params (Optional[Any], optional): The parameters to pass 
                to the function (default is None).
        
        Returns:
            Any: The return value of the function
        """
        contract = self._get_contract()
        function = getattr(contract.functions, function_name)

        return (
            function().call(block_identifier=block_identifier)
            if params is None
            else function(*params).call(block_identifier=block_identifier)
        )

In [6]:
ABIDIR = f"{DATA}/abi.json"
contract = FunctionCaller(address, w3, ABIDIR)
on_chain_data = contract.call_function("getReserves")
print(on_chain_data)

9888552049779


This number is consistent with the data on the Compound website but in different notation.

<img src="./fig/data.png" width="1400">